In [42]:
import pandas as pd
import requests
# import wget
import zipfile
from io import BytesIO
import os
from typing import List
import shutil


In [38]:
DATA_DIR = "./data/"

# List of all files we get from unzipping
all_files = ['NONDERIV_TRANS.tsv',
 'NONDERIV_HOLDING.tsv',
 'OWNER_SIGNATURE.tsv',
 'REPORTINGOWNER.tsv',
 'DERIV_HOLDING.tsv',
 'insider_transactions_metadata.json',
 'insider_transactions_readme.htm',
 'DERIV_TRANS.tsv',
 'SUBMISSION.tsv',
 'FOOTNOTES.tsv'
]

In [11]:
pd.set_option("display.max_columns", 100)

In [53]:
def get_insider_data_quaterly(year: int, quarter: int, data_dir:str = DATA_DIR) -> str:
    """
    Returns the folder name where the data is unzipped
    """

    base_url = f'https://www.sec.gov/files/structureddata/data/insider-transactions-data-sets/{str(year)}q{str(quarter)}_form345.zip'
    retval = requests.get(url=base_url)

    file_name = base_url.split('/')[-1]
    folder_name = file_name.split('.')[0]
    full_path = data_dir + file_name

    # Download the zip file in the data_dir
    with open(full_path, 'wb') as output_file:
        output_file.write(retval.content)
    
    # Create folder to extract files
    os.makedirs(os.path.join(data_dir, folder_name))
    
    try:
        # Extract the zip-files
        with zipfile.ZipFile(f'{full_path}', 'r') as zip_ref:
            zip_ref.extractall(os.path.join(data_dir, folder_name))
    except zipfile.BadZipFile as e:
        os.unlink(full_path)
        shutil.rmtree(os.path.join(data_dir, folder_name))
        print(f"data doesn't exist for {year} : {quarter}" )
        return ""

    
    return folder_name

In [40]:
def get_insider_data_from_files(interested_folders: List[str], interested_files: List[str] = all_files, data_dir:str = DATA_DIR) -> dict:
    d_data = {f:pd.DataFrame() for f in interested_files}

    for folder in interested_folders:
        for file in interested_files:
            if os.path.isfile(data_dir + folder + "/" + file):
                tmp_df = pd.read_csv(data_dir + folder + "/" + file, sep='\t', low_memory=False)
                d_data[file] = pd.concat([d_data[file], tmp_df])

    return d_data

In [55]:
years = [2023]
quarters = [4]
interested_files = ['NONDERIV_TRANS.tsv', 'NONDERIV_HOLDING.tsv',]
lst_folders = []

for year in years:
    for quar in quarters:
        lst_folders.append(get_insider_data_quaterly(year=year, quarter=quar, data_dir=DATA_DIR))

data doesn't exist for 2023 : 4


In [56]:
lst_folders

[None]

In [35]:
all_data_dict = get_insider_data_from_files(interested_files=interested_files, interested_folders=lst_folders)

/Users/kartikeysinha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (7,13,23) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/Users/kartikeysinha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (7,23) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/Users/kartikeysinha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/Users/kartikeysinha/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (13,23) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


In [46]:
all_data_dict['NONDERIV_TRANS.tsv']

,ACCESSION_NUMBER,NONDERIV_TRANS_SK,SECURITY_TITLE,SECURITY_TITLE_FN,TRANS_DATE,TRANS_DATE_FN,DEEMED_EXECUTION_DATE,DEEMED_EXECUTION_DATE_FN,TRANS_FORM_TYPE,TRANS_CODE,EQUITY_SWAP_INVOLVED,EQUITY_SWAP_TRANS_CD_FN,TRANS_TIMELINESS,TRANS_TIMELINESS_FN,TRANS_SHARES,TRANS_SHARES_FN,TRANS_PRICEPERSHARE,TRANS_PRICEPERSHARE_FN,TRANS_ACQUIRED_DISP_CD,TRANS_ACQUIRED_DISP_CD_FN,SHRS_OWND_FOLWNG_TRANS,SHRS_OWND_FOLWNG_TRANS_FN,VALU_OWND_FOLWNG_TRANS,VALU_OWND_FOLWNG_TRANS_FN,DIRECT_INDIRECT_OWNERSHIP,DIRECT_INDIRECT_OWNERSHIP_FN,NATURE_OF_OWNERSHIP,NATURE_OF_OWNERSHIP_FN
0,0001104659-21-045073,47618,"Common stock, par value $0.0001 per share",NaN,29-MAR-2021,NaN,NaN,NaN,4,J,0,F1,NaN,NaN,31250.00,NaN,0.00,F1,D,NaN,5450001.00,NaN,NaN,NaN,D,F2,NaN,NaN
1,0001104659-21-045074,33206,"Common stock, par value $0.0001 per share",NaN,29-MAR-2021,NaN,NaN,NaN,4,J,0,F1,NaN,NaN,31250.00,NaN,0.00,F1,D,NaN,5450001.00,NaN,NaN,NaN,I,NaN,See FN,F2
2,0001104659-21-045075,48937,"Common stock, par value $0.0001 per share",NaN,29-MAR-2021,NaN,NaN,NaN,4,J,0,F1,NaN,NaN,31250.00,NaN,0.00,F1,D,NaN,5450001.00,NaN,NaN,NaN,I,NaN,See FN,F2
3,0001104659-21-045076,51699,"Common stock, par value $0.0001 per share",NaN,29-MAR-2021,NaN,NaN,NaN,4,J,0,F1,NaN,NaN,31250.00,NaN,0.00,F1,D,NaN,5450001.00,NaN,NaN,NaN,I,NaN,See FN,F2
4,0000899243-21-014303,41168,Common Stock,NaN,26-MAR-2021,NaN,NaN,NaN,4,X,0,NaN,NaN,NaN,884175.00,NaN,0.01,NaN,A,NaN,2492342.00,NaN,NaN,NaN,I,NaN,"By Pillar Pharmaceuticals 6, L.P. See Footnotes","F2, F3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69769,0000886346-22-000077,6060986,Common Stock,NaN,31-MAR-2022,NaN,NaN,NaN,4,F,0,NaN,NaN,NaN,660.00,NaN,195.31,NaN,D,NaN,32749.00,NaN,NaN,NaN,D,NaN,NaN,NaN
69770,0001654954-22-004385,6060841,Community Bancorp. Common Stock,NaN,31-MAR-2022,NaN,NaN,NaN,4,P,false,NaN,NaN,NaN,250.00,NaN,22.75,NaN,A,NaN,35434.00,NaN,NaN,NaN,D,NaN,NaN,NaN
69771,0001562180-22-003046,6060299,Common Stock,NaN,31-MAR-2022,NaN,NaN,NaN,4,S,false,F1,NaN,NaN,4500.00,NaN,29.24,F2,D,NaN,26022.00,F3,NaN,NaN,D,NaN,NaN,NaN
69772,0000899243-22-013011,6060361,Class I common shares of beneficial interest,NaN,31-MAR-2022,NaN,NaN,NaN,4,S,0,NaN,NaN,NaN,164466.47,F1,9.12,F1,D,NaN,211788.28,F1,NaN,NaN,D,NaN,NaN,NaN


In [39]:
all_data_dict['NONDERIV_HOLDING.tsv']

,ACCESSION_NUMBER,NONDERIV_HOLDING_SK,SECURITY_TITLE,SECURITY_TITLE_FN,TRANS_FORM_TYPE,TRANS_FORM_TYPE_FN,SHRS_OWND_FOLWNG_TRANS,SHRS_OWND_FOLWNG_TRANS_FN,VALU_OWND_FOLWNG_TRANS,VALU_OWND_FOLWNG_TRANS_FN,DIRECT_INDIRECT_OWNERSHIP,DIRECT_INDIRECT_OWNERSHIP_FN,NATURE_OF_OWNERSHIP,NATURE_OF_OWNERSHIP_FN
0,0000899243-21-014303,12020,Common Stock,NaN,NaN,NaN,386110.0,NaN,NaN,NaN,I,NaN,By Abude Umari See Footnotes,"F5, F6, F2"
1,0000899243-21-014303,12021,Common Stock,NaN,NaN,NaN,26308.0,NaN,NaN,NaN,I,NaN,By Youssef El Zein See Footnotes,F2
2,0000899243-21-014294,17299,Class A Common Stock,NaN,NaN,NaN,5750000.0,"F4, F3",NaN,NaN,I,NaN,See Footnotes,"F4, F3"
3,0001854475-21-000002,10067,"Class A Common Stock, par value $0.001",F1,NaN,NaN,16029.0,NaN,NaN,NaN,D,NaN,NaN,NaN
4,0001082440-21-000006,39251,Common Stock,NaN,NaN,NaN,53872.0,F5,NaN,NaN,D,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22336,0001185185-22-000385,2014944,"Common Stock, $5.00 par value per share",NaN,NaN,NaN,1381227.0,NaN,NaN,NaN,I,NaN,"By the Article 6 Marital Trust, under the Jerr...",NaN
22337,0000806279-22-000069,2014723,Common Stock,NaN,NaN,NaN,6037.0,NaN,NaN,NaN,I,NaN,Jack Giambalvo Motor Co profit sharing plan FB...,NaN
22338,0001290476-22-000014,2014869,Common Stock,NaN,NaN,NaN,264.0,F1,NaN,NaN,D,NaN,NaN,NaN
22339,0001127602-22-010778,2014840,Common Stock,NaN,NaN,NaN,2435.0,NaN,NaN,NaN,I,NaN,Prft Shring/Saving Plan Trust,NaN


In [43]:
def cleanup(data_dir:str = DATA_DIR) -> None:
    for filename in os.listdir(data_dir):
        file_path = os.path.join(data_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
    return

In [44]:
cleanup()

In [25]:
deriv_transactions = pd.read_csv('DERIV_TRANS.tsv', sep='\t')
print(deriv_transactions.info())
deriv_transactions.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29457 entries, 0 to 29456
Data columns (total 42 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ACCESSION_NUMBER              29457 non-null  object 
 1   DERIV_TRANS_SK                29457 non-null  int64  
 2   SECURITY_TITLE                29457 non-null  object 
 3   SECURITY_TITLE_FN             1970 non-null   object 
 4   CONV_EXERCISE_PRICE           14466 non-null  float64
 5   CONV_EXERCISE_PRICE_FN        16987 non-null  object 
 6   TRANS_DATE                    29457 non-null  object 
 7   TRANS_DATE_FN                 275 non-null    object 
 8   DEEMED_EXECUTION_DATE         154 non-null    object 
 9   DEEMED_EXECUTION_DATE_FN      30 non-null     object 
 10  TRANS_FORM_TYPE               29457 non-null  int64  
 11  TRANS_CODE                    29457 non-null  object 
 12  EQUITY_SWAP_INVOLVED          29457 non-null  object 
 13  E

/Users/kartikeysinha/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ACCESSION_NUMBER,DERIV_TRANS_SK,SECURITY_TITLE,SECURITY_TITLE_FN,CONV_EXERCISE_PRICE,CONV_EXERCISE_PRICE_FN,TRANS_DATE,TRANS_DATE_FN,DEEMED_EXECUTION_DATE,DEEMED_EXECUTION_DATE_FN,TRANS_FORM_TYPE,TRANS_CODE,EQUITY_SWAP_INVOLVED,EQUITY_SWAP_TRANS_CD_FN,TRANS_TIMELINESS,TRANS_TIMELINESS_FN,TRANS_SHARES,TRANS_SHARES_FN,TRANS_TOTAL_VALUE,TRANS_TOTAL_VALUE_FN,TRANS_PRICEPERSHARE,TRANS_PRICEPERSHARE_FN,TRANS_ACQUIRED_DISP_CD,TRANS_ACQUIRED_DISP_CD_FN,EXCERCISE_DATE,EXCERCISE_DATE_FN,EXPIRATION_DATE,EXPIRATION_DATE_FN,UNDLYNG_SEC_TITLE,UNDLYNG_SEC_TITLE_FN,UNDLYNG_SEC_SHARES,UNDLYNG_SEC_SHARES_FN,UNDLYNG_SEC_VALUE,UNDLYNG_SEC_VALUE_FN,SHRS_OWND_FOLWNG_TRANS,SHRS_OWND_FOLWNG_TRANS_FN,VALU_OWND_FOLWNG_TRANS,VALU_OWND_FOLWNG_TRANS_FN,DIRECT_INDIRECT_OWNERSHIP,DIRECT_INDIRECT_OWNERSHIP_FN,NATURE_OF_OWNERSHIP,NATURE_OF_OWNERSHIP_FN
0,0000899243-23-016780,2413204,PUT Option (Obligation to Buy),NaN,4.0,NaN,28-JUN-2023,NaN,NaN,NaN,4,X,0,NaN,NaN,NaN,20700.0,NaN,NaN,NaN,0.0,NaN,D,NaN,31-JAN-2023,NaN,19-JAN-2024,NaN,Common Stock,NaN,20700.0,NaN,NaN,NaN,1281100.0,NaN,NaN,NaN,I,NaN,By Crescent River LLC,NaN
1,0000899243-23-016780,2413206,PUT Option (Obligation to Buy),NaN,3.5,NaN,28-JUN-2023,NaN,NaN,NaN,4,X,0,NaN,NaN,NaN,15300.0,NaN,NaN,NaN,0.0,NaN,D,NaN,17-NOV-2022,NaN,19-JAN-2024,NaN,Common Stock,NaN,15300.0,NaN,NaN,NaN,102300.0,NaN,NaN,NaN,I,NaN,By Elstead 2020 GRAT,NaN
2,0000899243-23-016780,2413205,PUT Option (Obligation to Buy),NaN,3.5,NaN,28-JUN-2023,NaN,NaN,NaN,4,X,0,NaN,NaN,NaN,43900.0,NaN,NaN,NaN,0.0,NaN,D,NaN,03-NOV-2022,NaN,19-JAN-2024,NaN,Common Stock,NaN,43900.0,NaN,NaN,NaN,1237200.0,NaN,NaN,NaN,I,NaN,By Crescent River LLC,NaN
3,0000899243-23-016780,2413209,PUT Option (Obligation to Buy),NaN,4.0,NaN,28-JUN-2023,NaN,NaN,NaN,4,X,0,NaN,NaN,NaN,100000.0,NaN,NaN,NaN,0.0,NaN,D,NaN,25-JAN-2023,NaN,19-JAN-2024,NaN,Common Stock,NaN,100000.0,NaN,NaN,NaN,143600.0,NaN,NaN,NaN,I,NaN,By Shiraz Capital LLC,NaN
4,0000899243-23-016780,2413208,PUT Option (Obligation to Buy),NaN,4.0,NaN,28-JUN-2023,NaN,NaN,NaN,4,X,0,NaN,NaN,NaN,31400.0,NaN,NaN,NaN,0.0,NaN,D,NaN,12-DEC-2022,NaN,19-JAN-2024,NaN,Common Stock,NaN,31400.0,NaN,NaN,NaN,243600.0,NaN,NaN,NaN,I,NaN,By Shiraz Capital LLC,NaN


In [27]:
non_deriv_transactions = pd.read_csv('NONDERIV_TRANS.tsv', sep='\t')
print(non_deriv_transactions.info())
non_deriv_transactions.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68588 entries, 0 to 68587
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ACCESSION_NUMBER              68588 non-null  object 
 1   NONDERIV_TRANS_SK             68588 non-null  int64  
 2   SECURITY_TITLE                68588 non-null  object 
 3   SECURITY_TITLE_FN             2206 non-null   object 
 4   TRANS_DATE                    68588 non-null  object 
 5   TRANS_DATE_FN                 418 non-null    object 
 6   DEEMED_EXECUTION_DATE         689 non-null    object 
 7   DEEMED_EXECUTION_DATE_FN      53 non-null     object 
 8   TRANS_FORM_TYPE               68588 non-null  int64  
 9   TRANS_CODE                    68588 non-null  object 
 10  EQUITY_SWAP_INVOLVED          68588 non-null  object 
 11  EQUITY_SWAP_TRANS_CD_FN       16157 non-null  object 
 12  TRANS_TIMELINESS              672 non-null    object 
 13  T

/Users/kartikeysinha/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (7,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ACCESSION_NUMBER,NONDERIV_TRANS_SK,SECURITY_TITLE,SECURITY_TITLE_FN,TRANS_DATE,TRANS_DATE_FN,DEEMED_EXECUTION_DATE,DEEMED_EXECUTION_DATE_FN,TRANS_FORM_TYPE,TRANS_CODE,EQUITY_SWAP_INVOLVED,EQUITY_SWAP_TRANS_CD_FN,TRANS_TIMELINESS,TRANS_TIMELINESS_FN,TRANS_SHARES,TRANS_SHARES_FN,TRANS_PRICEPERSHARE,TRANS_PRICEPERSHARE_FN,TRANS_ACQUIRED_DISP_CD,TRANS_ACQUIRED_DISP_CD_FN,SHRS_OWND_FOLWNG_TRANS,SHRS_OWND_FOLWNG_TRANS_FN,VALU_OWND_FOLWNG_TRANS,VALU_OWND_FOLWNG_TRANS_FN,DIRECT_INDIRECT_OWNERSHIP,DIRECT_INDIRECT_OWNERSHIP_FN,NATURE_OF_OWNERSHIP,NATURE_OF_OWNERSHIP_FN
0,0000899243-23-016780,6418208,COMMON STOCK,NaN,28-JUN-2023,NaN,NaN,NaN,4,X,0,NaN,NaN,NaN,131400.0,NaN,4.0,NaN,A,NaN,6745038.0,NaN,NaN,NaN,I,NaN,By Shiraz Capital LLC,NaN
1,0000899243-23-016780,6418207,COMMON STOCK,NaN,28-JUN-2023,NaN,NaN,NaN,4,X,0,NaN,NaN,NaN,24800.0,NaN,4.0,NaN,A,NaN,504500.0,NaN,NaN,NaN,I,NaN,By Havana 2020 GRAT,NaN
2,0000899243-23-016780,6418206,COMMON STOCK,NaN,28-JUN-2023,NaN,NaN,NaN,4,X,0,NaN,NaN,NaN,15300.0,NaN,3.5,NaN,A,NaN,897700.0,NaN,NaN,NaN,I,NaN,By Elstead 2020 GRAT,NaN
3,0000899243-23-016780,6418205,COMMON STOCK,NaN,28-JUN-2023,NaN,NaN,NaN,4,X,0,NaN,NaN,NaN,43900.0,NaN,3.5,NaN,A,NaN,8042300.0,NaN,NaN,NaN,I,NaN,By Crescent River LLC,NaN
4,0000899243-23-016780,6418204,COMMON STOCK,NaN,28-JUN-2023,NaN,NaN,NaN,4,X,0,NaN,NaN,NaN,20700.0,NaN,4.0,NaN,A,NaN,7998400.0,NaN,NaN,NaN,I,NaN,By Crescent River LLC,NaN


In [28]:
owner_signature = pd.read_csv('OWNER_SIGNATURE.tsv', sep='\t')
print(owner_signature.info())
owner_signature.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59339 entries, 0 to 59338
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ACCESSION_NUMBER    59339 non-null  object
 1   OWNERSIGNATURENAME  59339 non-null  object
 2   OWNERSIGNATUREDATE  59339 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB
None


,ACCESSION_NUMBER,OWNERSIGNATURENAME,OWNERSIGNATUREDATE
0,0000899243-23-016780,Brad Nelson as Attorney in Fact for Farhad Fre...,30-JUN-2023
1,0001433195-23-000087,/s/ Matthew Mazza as Attorney-in-Fact for Andr...,30-JUN-2023
2,0001433195-23-000086,"/s/ Matthew Mazza, as Attorney-in-Fact for Win...",30-JUN-2023
3,0001569187-23-000087,"/s/ Louis S. Haddad, as Attorney-in-Fact for D...",30-JUN-2023
4,0001433195-23-000085,/s/ Matthew S Mazza as Attorney-in-Fact for Ol...,30-JUN-2023


In [30]:
owner_reporting = pd.read_csv('REPORTINGOWNER.tsv', sep='\t')
print(owner_reporting.info())
owner_reporting.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60049 entries, 0 to 60048
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ACCESSION_NUMBER       60049 non-null  object
 1   RPTOWNERCIK            60049 non-null  int64 
 2   RPTOWNERNAME           60049 non-null  object
 3   RPTOWNER_RELATIONSHIP  60043 non-null  object
 4   RPTOWNER_TITLE         23616 non-null  object
 5   RPTOWNER_TXT           1538 non-null   object
 6   RPTOWNER_STREET1       59820 non-null  object
 7   RPTOWNER_STREET2       35784 non-null  object
 8   RPTOWNER_CITY          59993 non-null  object
 9   RPTOWNER_STATE         59868 non-null  object
 10  RPTOWNER_ZIPCODE       59764 non-null  object
 11  RPTOWNER_STATE_DESC    2246 non-null   object
 12  FILE_NUMBER            60049 non-null  object
dtypes: int64(1), object(12)
memory usage: 6.0+ MB
None


,ACCESSION_NUMBER,RPTOWNERCIK,RPTOWNERNAME,RPTOWNER_RELATIONSHIP,RPTOWNER_TITLE,RPTOWNER_TXT,RPTOWNER_STREET1,RPTOWNER_STREET2,RPTOWNER_CITY,RPTOWNER_STATE,RPTOWNER_ZIPCODE,RPTOWNER_STATE_DESC,FILE_NUMBER
0,0000899243-23-016780,1071351,EBRAHIMI FARHAD FRED,TenPercentOwner,NaN,NaN,"191 UNIVERSITY BLVD, SUITE 246",NaN,DENVER,CO,80206,NaN,001-37725
1,0001433195-23-000087,1643507,von Blottnitz Andreas,Director,NaN,NaN,70 CASTILIAN DRIVE,NaN,SANTA BARBARA,CA,93117,NaN,001-37468
2,0001433195-23-000086,1439919,Webb Winifred Markus,Director,NaN,NaN,70 CASTILIAN DRIVE,NaN,SANTA BARBARA,CA,93117,NaN,001-37468
3,0001569187-23-000087,1351710,Hoffler Daniel A,Director,NaN,NaN,"C/O ARMADA HOFFLER PROPERTIES, INC.","222 CENTRAL PARK AVENUE, SUITE 2100",VIRGINIA BEACH,VA,23462,NaN,001-35908
4,0001433195-23-000085,1801732,Nottebohm Olivia,Director,NaN,NaN,70 CASTILIAN DR.,NaN,SANTA BARBARA,CA,93117,NaN,001-37468


In [31]:
submissions = pd.read_csv('REPORTINGOWNER.tsv', sep='\t')
print(submissions.info())
submissions.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60049 entries, 0 to 60048
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ACCESSION_NUMBER       60049 non-null  object
 1   RPTOWNERCIK            60049 non-null  int64 
 2   RPTOWNERNAME           60049 non-null  object
 3   RPTOWNER_RELATIONSHIP  60043 non-null  object
 4   RPTOWNER_TITLE         23616 non-null  object
 5   RPTOWNER_TXT           1538 non-null   object
 6   RPTOWNER_STREET1       59820 non-null  object
 7   RPTOWNER_STREET2       35784 non-null  object
 8   RPTOWNER_CITY          59993 non-null  object
 9   RPTOWNER_STATE         59868 non-null  object
 10  RPTOWNER_ZIPCODE       59764 non-null  object
 11  RPTOWNER_STATE_DESC    2246 non-null   object
 12  FILE_NUMBER            60049 non-null  object
dtypes: int64(1), object(12)
memory usage: 6.0+ MB
None


,ACCESSION_NUMBER,RPTOWNERCIK,RPTOWNERNAME,RPTOWNER_RELATIONSHIP,RPTOWNER_TITLE,RPTOWNER_TXT,RPTOWNER_STREET1,RPTOWNER_STREET2,RPTOWNER_CITY,RPTOWNER_STATE,RPTOWNER_ZIPCODE,RPTOWNER_STATE_DESC,FILE_NUMBER
0,0000899243-23-016780,1071351,EBRAHIMI FARHAD FRED,TenPercentOwner,NaN,NaN,"191 UNIVERSITY BLVD, SUITE 246",NaN,DENVER,CO,80206,NaN,001-37725
1,0001433195-23-000087,1643507,von Blottnitz Andreas,Director,NaN,NaN,70 CASTILIAN DRIVE,NaN,SANTA BARBARA,CA,93117,NaN,001-37468
2,0001433195-23-000086,1439919,Webb Winifred Markus,Director,NaN,NaN,70 CASTILIAN DRIVE,NaN,SANTA BARBARA,CA,93117,NaN,001-37468
3,0001569187-23-000087,1351710,Hoffler Daniel A,Director,NaN,NaN,"C/O ARMADA HOFFLER PROPERTIES, INC.","222 CENTRAL PARK AVENUE, SUITE 2100",VIRGINIA BEACH,VA,23462,NaN,001-35908
4,0001433195-23-000085,1801732,Nottebohm Olivia,Director,NaN,NaN,70 CASTILIAN DR.,NaN,SANTA BARBARA,CA,93117,NaN,001-37468
